In [47]:
import pandas as pd 
import numpy as np 
import random
import matplotlib.pyplot as plt 
from math import sqrt
%matplotlib qt

In [48]:
np.random.seed(100)

In [49]:
data = pd.read_excel('15-Points.xlsx')
data

,x,y,City
0,5.500000e-08,9.860000e-09,1
1,-2.887330e+01,-7.980000e-08,2
2,-7.929160e+01,-2.140330e+01,3
3,-1.465770e+01,-4.338960e+01,4
4,-6.474730e+01,2.189820e+01,5
5,-2.905850e+01,-4.321670e+01,6
6,-7.207850e+01,1.815810e-01,7
7,-3.603660e+01,-2.161350e+01,8
8,-5.048080e+01,7.374470e+00,9
9,-5.058590e+01,-2.158820e+01,10


In [50]:
def load_data(data):
  data = [{'x' : x[0], 'y' : x[1], 'city' : x[2]} for x in data.values]
  return data

In [51]:
def Generate_distance(data):
    n = len(data)
    dist_matrix = np.zeros((n, n))
    for i in range(n-1):
        x1 = data[i]['x']
        y1 = data[i]['y']
        dist_matrix[i, i] = np.inf
        for j in range(i + 1, n):
            x2 = data[j]['x']
            y2 = data[j]['y']
            dist_matrix[i, j] = sqrt((x1 - x2)**2 + (y1 - y2)**2)
            dist_matrix[j, i] = dist_matrix[i, j]
    dist_matrix[i + 1, i + 1] = np.inf
    return dist_matrix

In [52]:
gen_count=100
pop_size = 50 
Elitism_percentage = 0.05
Crossover_robability = 0.8
Mutation_probability = 0.15

In [53]:
Population_size = 50
Elitism_percentage = 0.05
CrossOver_Prop = 0.8
Mutation_Prop = 0.15
Generation_count = 150

In [54]:
def initialize_genetic(num_of_cities):
    pop = [np.random.permutation(num_of_cities) for i in range (pop_size)]
    return np.array(pop)

In [55]:
def Cost_Function(population, distances):
  J = np.zeros((len(population)))
  index = 0
  for arr in population:
    dist = 0
    c0 = arr[0]
    for i in range(len(arr) - 1):
      c1 = arr[i]
      c2 = arr[i + 1]
      dist += distances[c1, c2]

    dist += distances[c0, c2]
    J[index] = dist
    index += 1
  return J

In [56]:
def Total_distance(arr, distances):
  dist = 0
  c0 = arr[0]
  for i in range(len(arr) - 1):
      c1 = arr[i]
      c2 = arr[i + 1]
      dist += distances[c1, c2]

  dist += distances[c0, c2]
  return dist

In [57]:
def Elitism(J):
  return population(argmin(J))

In [58]:
def CrossOver_SinglePoint(parent1, parent2):
    arr_size = len(parent1)//2
    
    arr1_1 = parent1[:arr_size]
    arr1_2 = parent1[arr_size:]
    arr2_1 = parent2[:arr_size]
    arr2_2 = parent2[arr_size:]

    child1 = np.concatenate((arr1_1, arr2_2))
    child2 = np.concatenate((arr2_1, arr1_2))
    return child1 , child2

In [59]:
def CrossOver_MultiPoint(parent1, parent2):
    arr_size = len(parent1)//3

    arr1_1 = parent1[:arr_size]
    arr1_2 = parent1[arr_size:(arr_size*2)]
    arr1_3 = parent1[(arr_size*2):]
    arr2_1 = parent2[:arr_size]
    arr2_2 = parent2[arr_size:(arr_size*2)]
    arr2_3 = parent2[(arr_size*2):]

    child1 = np.concatenate((arr1_1, arr2_2,arr1_3))
    child2 = np.concatenate((arr2_1, arr1_2,arr2_3)) 
    return child1 , child2

In [60]:
def CrossOver_Uniform(parent1, parent2):

    child1 = [None] * len(parent1)
    child2 = [None] * len(parent2)

    for i in range(len(parent1)):
        # randomly select a gene from one of the parents
        if random.random() < 0.5:
            child1[i] = parent1[i]
            child2[i] = parent2[i]
        else:
            child1[i] = parent2[i]
            child2[i] = parent1[i]

    return child1 , child2

In [61]:
def modified_uniform_crossover(Chromosome1, Chromosome2):    
    # create a mask to determine which cities to copy from Chromosome1
    mask = [np.random.choice([0, 1]) for _ in range(len(Chromosome1))]
    
    # create a set of cities already copied from Chromosome1
    copied_cities = [Chromosome1[i] for i in range(len(Chromosome1)) if mask[i] == 1]
    
    # create a list of remaining cities in the order of Chromosome2
    remaining_cities = [city for city in Chromosome2 if city not in copied_cities]
    
    # create the offspring by combining the copied cities and remaining cities
    offspring = [None] * len(Chromosome1)
    for i in range(len(Chromosome1)):
        if mask[i] == 1:
            offspring[i] = Chromosome1[i]
        else:
            offspring[i] = remaining_cities.pop(0)
    
    return offspring

In [62]:
def Swap_Mutation(parent):
    child = parent.copy()
    idx = range(len(child))
    i1, i2 = random.sample(idx, 2)
    temp = child[i1]
    child[i1] = child[i2]
    child[i2] = temp
    return child

In [63]:
def Scramble_Mutation(parent):
    child = parent.copy()
    index1 = np.random.randint(len(child) - 1)
    index2 = np.random.randint(len(child) - 1)
    
    if index1 > index2:
        index1, index2 = index2, index1

    subset = child[index1:index2+1]
    np.random.shuffle(subset)
    child[index1: index2 + 1] = subset
        
    return list(child)

In [64]:
def Inversion_Mutation(parent):
    child = parent.copy()
    index1 = np.random.randint(len(child) - 1)
    index2 = np.random.randint(len(child) - 1)

    if index1 > index2:
        index1, index2 = index2, index1
    
    subset = child[index1: index2 + 1]
    inverted_values = np.flip(subset)
    child[index1: index2 + 1] = inverted_values
        
    return list(child)

In [65]:
def TSP(data, Best_solution):
    x = [x['x'] for x in data]
    y = [x['y'] for x in data]
    plt.scatter(x, y, color = 'b')
    x_init, y_init = data[Best_solution[0]]['x'], data[Best_solution[0]]['y']
    plt.scatter(x_init, y_init, color = 'r')
    plt.text(x_init - 3, y_init - 4, "Start")

    x = []
    y = []
    x.append(x_init)
    y.append(y_init)
    TSP_plot = plt.plot(x, y, color = 'k')

    for i, city in enumerate(Best_solution[0:]):
        city_x, city_y = data[Best_solution[i]]['x'], data[Best_solution[i]]['y']
        x.append(city_x)
        y.append(city_y)  
        plt.setp(TSP_plot, xdata = x, ydata = y)
        plt.pause(0.3)

    x.append(x_init)
    y.append(y_init)
    plt.setp(TSP_plot, xdata = x, ydata = y)
    plt.pause(0.3)

In [66]:
def Genetic_Algorithm (data,Elitism_percentage,Crossover_robability,Mutation_probability,gen_count):
    
    # Load Data
    data = load_data(data)

    # Calculate the distances and intialize population
    distances = Generate_distance(data)
    population = initialize_genetic(len(data))

    # calculate Cost 
    J = Cost_Function(population, distances) 
    # sort population
    sorted_population = population[J.argsort()]

    # Calculate the number of elites, crossovers and mutations
    num_elites = round(Elitism_percentage * len(sorted_population))
    num_crossovers = round(Crossover_robability * (len(sorted_population) - num_elites))
    num_mutations = round(Mutation_probability * (len(population) - num_elites))


    for gene in range(gen_count):
        Next_Generation = []
             
        for i in range(num_elites):
          Next_Generation.append(sorted_population[i])

        for i in range(num_crossovers):
          inx1 = np.random.randint(num_elites, len(sorted_population) - 1)
          inx2 = np.random.randint(num_elites, len(sorted_population) - 1)
          parent1 = sorted_population[inx1]
          parent2 = sorted_population[inx2]

          child =  modified_uniform_crossover(parent1, parent2)
          Next_Generation.append(child)

        for i in range(num_mutations):
          inx = np.random.randint(len(sorted_population) - 1)
          parent = sorted_population[inx]

          Mutation_method = np.random.choice(['Swap_Mutation', 'Scramble_Mutation', 'Inversion_Mutation'])
          if Mutation_method == 'Swap_Mutation':
            child = Swap_Mutation(parent)

          elif Mutation_method == 'Inversion_Mutation':
            child = Inversion_Mutation(parent)

          else:
            child = Scramble_Mutation(parent)

          selection = [parent, child]
          Next_Generation.append(child)
        
        # Set the current population to the next generation
        Next_Generation = np.concatenate([Next_Generation, sorted_population]).reshape(-1, len(distances))

        # Calculate the cost function for each population
        J = Cost_Function(Next_Generation, distances)
        # Sort the population by J value
        sorted_population = Next_Generation[J.argsort()][:50]

    # Return the best solution found
    Best_solution = sorted_population[0]
    
    distance = Total_distance(Best_solution, distances)
    TSP(data, Best_solution)


    return Best_solution ,distance

In [67]:
BestSolution,distance = Genetic_Algorithm(data, Elitism_percentage, Crossover_robability, Mutation_probability, gen_count)

In [68]:
BestSolution

array([ 5,  3, 10,  0, 12, 14,  8,  4,  6,  2, 11, 13,  9,  1,  7])

In [69]:
print(f'the total distance = {distance}')

the total distance = 315.25004166379216
